In [1]:
import os
import numpy as np
import re

# XmlParser

In [2]:
def avg(file_name):
    SumAccum = 0
    filtered = ''.join(filter(lambda x: x not in '".,;!-\n',file_name))
    for ch in filtered.split():
        character = len(ch)
        SumAccum = SumAccum + character
    average = (SumAccum) /(float) (len(filtered.split()))
    average=round(average)
    return average


In [3]:
#import os
from xml.dom.minidom import parse
import xml.dom.minidom

def get_data(dataFolder, testFile):
    global urlcount,avg_nowords
    xmlFile = os.path.join(dataFolder, testFile)
    ## creating document object model from the xml file
    DOMTree = xml.dom.minidom.parse(xmlFile)
    collection = DOMTree.documentElement
    ## getting author ID of the xml file 
    authorID = testFile.replace('.xml', '')
    ## getting language of the xml file 
    Language = collection.getAttribute("lang")
    ## getting data in the xml file
    Documents = collection.getElementsByTagName("document")
    Data = []
    for Document in Documents:
        if len(Document.childNodes) > 0:
            Data.append(Document.childNodes[0].data.encode('utf-8'))
            Datastr= ''.join(Data)
            #urlcount=Datastr.count('http')
            data1 = re.sub(r'/\S+','https',(' '.join(map(str,(Datastr.split('http://t.co'))))))
            data = re.sub(r'http\S+','https',data1)
            data = re.sub(r'@\S+','@',data)
            data=re.sub(r'#\S+','#',data)
            data=re.sub(r'\.+', ". ", data)
            data=re.sub(r'\!+', ". ", data)
            data = re.sub(r'-',r'',data)
            data =data.lower()
            #data = re.sub(r'[:)|;|,|!|?|*]',r'',data)
            Data= re.sub(r'\xf0\x9f\S+','emoji',data)
            avg_nowords=avg(Data)
        else:
            Data.append(' ')
    return Data, Language, authorID, avg_nowords#, urlcount,atcount

In [4]:
def load_data(dataFolder, Action):
    truthFilePath = os.path.join(dataFolder, 'truth.txt')
    
    '''specific for training data '''
    genderList = []     ## creating empty list for appending gender class information
    languageList = []   ## creating empty list for appending language of the xml file  
    authorIDList = []   ## creating empty list for appending author ID of the xml file
    authorTextList = [] ## creating empty list for appending author's tweets from the xml file 
    avglist=[]
    # httpList = []
    
    if Action == 'TRAIN':   ## loop for loading training data
        File = open(truthFilePath, 'r+') ## reading truth file
        Text = File.read()
        File.close()
        truthTexts = Text.split('\n')
        for truthText in truthTexts: ## loop for loading tweets of the author and classes from the xml files in the truth file
            truthText = truthText.split(':::')
            if len(truthText) == 2:
                xmlFile = truthText[0] + '.xml'
                Data, Language, authorID,avg_nowords = get_data(dataFolder, xmlFile)
                genderList.append(truthText[1])
                authorTextList.append(Data)
                languageList.append(Language)
                #httpList.append(urlcount)
                authorIDList.append(authorID)
                avglist.append(avg_nowords)
            else:
                pass
    else:                          ## loop for loading test data
        xmlFiles = os.listdir(dataFolder)
        for xmlFile in xmlFiles:   ## loop for loading tweets of the author from the xml files in the data folder
            Data, Language, authorID,avg_nowords= get_data(dataFolder, xmlFile)
            authorTextList.append(Data)
            languageList.append(Language)
            authorIDList.append(authorID)
            avglist.append(avg_nowords)
    return genderList, languageList, authorIDList, authorTextList, avglist#, httpList, atList


## Calling Training Data

In [5]:
#modelFoldertrain = 'Modeltr/'#folder where system model will be stored
trainFolder = 'rusprofiling.training.20170629/'# the xml files have be saved with the ground label file labeled as "truth.txt"
dataFolder = trainFolder
genderList_train, languageList_train, authorIDList_train, authorTextList_train,avglist_train = load_data(dataFolder, 'TRAIN')
tempTextList_train = [''.join(authorText) for authorText in authorTextList_train ]



## Calling Testing Data

In [6]:
#modelFolder = 'Modelte/'#folder where system model will be stored
#trainFolder = 'rusprofiling.training.20170629/'# the xml files have be saved with the ground label file labeled as "truth.txt"
testFolder = 'rustesting corpus/rusprofiling.test.3.20170629'
#dataFolder = trainFolder
dataFolder = testFolder
genderList_test, languageList_test, authorIDList_test, authorTextList_test,avglist_test= load_data(dataFolder, 'TEST')
tempTextList_test = [''.join(authorText) for authorText in authorTextList_test ]

# FeatureLearner

In [7]:
## importing required library for feature extraction

from sklearn.externals import joblib
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
    
## function for creating document term matrix
def get_tdm(data_samples,tdata, Language, Action, mini_df):
    if Action == 'TRAIN':
        tf_vectorizer = CountVectorizer(min_df = mini_df,ngram_range=(1,2))#,max_features = 30000)
        print 'term document matrix'
        print 'minimum document frequency', mini_df
        tf_vectorizer.fit(data_samples)
        dtMatrix = tf_vectorizer.transform(tdata)
        #  dtMatrix = tf_vectorizer.transform(data_samples)
    return dtMatrix

## function for creating term frequency - inverse document frequency matrix
def get_tdidf(data_samples,tdata, Language, Action, mini_df):
    if Action == 'TRAIN':
        print 'term document matrix'
        print 'minimum document frequency', mini_df
        tfidf_vectorizer = TfidfVectorizer(min_df = mini_df,ngram_range=(1,2),max_features = 20000)
        tfidf_vectorizer = tfidf_vectorizer.fit(data_samples)
        tfidfMatrix = tfidf_vectorizer.fit_transform(tdata)
        return tfidfMatrix


## Training Data Tesing for TDM

In [8]:
avglist_train_vec=(np.asarray(avglist_train)).reshape(-1,1)
avglist_test_vec=(np.asarray(avglist_test)).reshape(-1,1)

In [9]:
dtMatTrain_td = get_tdm(tempTextList_train+tempTextList_test,tempTextList_train,languageList_train[0],'TRAIN',3)
dtMatTrain_td_1=dtMatTrain_td.todense()
feat_vec_train =np.concatenate((dtMatTrain_td_1,avglist_train_vec),axis = 1)

term document matrix
minimum document frequency 3


In [10]:
feat_vec_train.shape

(600, 67891)

In [11]:
dtMatTest_td = get_tdm(tempTextList_train+tempTextList_test,tempTextList_test,languageList_test[0],'TRAIN',3)
dtMatTest_td_1=dtMatTest_td.todense()
feat_vec_test = np.concatenate((dtMatTest_td_1,avglist_test_vec),axis = 1)
feat_vec_train.shape, feat_vec_test.shape

term document matrix
minimum document frequency 3


((600, 67891), (400, 67891))

## Training Data Tesing for TFIDF

In [12]:
dtMatTrain = get_tdidf(tempTextList_train+tempTextList_test,tempTextList_train,languageList_train[0],'TRAIN',3)
dtMatTrain_1= dtMatTrain.todense()
feat_vec_train_2 =np.concatenate((dtMatTrain_1,avglist_train_vec),axis = 1)

term document matrix
minimum document frequency 3


In [13]:
dtMatTest = get_tdidf(tempTextList_train+tempTextList_test,tempTextList_test,languageList_test[0],'TRAIN',3)
dtMatTest_1= dtMatTest.todense()
feat_vec_test_2 =np.concatenate((dtMatTest_1,avglist_test_vec),axis = 1)

term document matrix
minimum document frequency 3


In [14]:
feat_vec_train_2.shape, feat_vec_test_2.shape

((600, 20001), (400, 19833))

In [15]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.decomposition import TruncatedSVD
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier,GradientBoostingClassifier,ExtraTreesClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LogisticRegression

In [16]:
#Cross-validation
#from sklearn.decomposition import NMF

def cv(Train_Mat , Target_label, classifier):
    tot1 = 0
    for i in range(1,11):
        #print i
        X_train,X_test,y_train,y_test = train_test_split(Train_Mat,Target_label,test_size=0.1,random_state=i)
        if classifier == 'SVM':
            clf_svm = svm.SVC(kernel = 'linear')
            clf_svm.fit(X_train,y_train)
            tot = clf_svm.score(X_test, y_test)
            #print 'i:',i,'tot:',tot
        if classifier == 'DT':
            clf_DT = DecisionTreeClassifier()
            clf_DT.fit(X_train,y_train)
            tot = clf_DT.score(X_test, y_test)
            #print 'i:',i,'tot:',tot
            #tot1 = tot1+tot
            #return (tot1/10)*100
        if classifier == 'Ada':
            clf_ada = AdaBoostClassifier(n_estimators=100)
            clf_ada.fit(X_train,y_train)
            tot = clf_ada.score(X_test, y_test)
            #print 'i:',i,'tot:',tot
            #tot1 = tot1+tot
            #return (tot1/10)*100
        if classifier == 'RF':
            clf_RF = RandomForestClassifier(n_estimators=100)
            clf_RF.fit(X_train,y_train)
            tot = clf_RF.score(X_test, y_test)
        if classifier == 'LR':
            clf_LR = LogisticRegression(penalty='l2')
            clf_LR.fit(X_train,y_train)
            tot = clf_LR.score(X_test,y_test)
            #print 'i:',i,'tot:',tot        
        tot1 = tot1+tot
    return (tot1/10.0)*100
        #print tot
        #tot1 = tot1+tot		

### SVM Classifier

In [18]:
cv(feat_vec_train,genderList_train,classifier='SVM')

79.833333333333329

In [19]:
cv(feat_vec_train_2,genderList_train,classifier='SVM')

77.5

### DecisionTree Classifier

In [20]:
cv(feat_vec_train,genderList_train,classifier='DT')

74.499999999999986

In [21]:
cv(feat_vec_train_2,genderList_train,classifier='DT')

67.166666666666657

### AdaBoost Classifier

In [33]:
cv(feat_vec_train,genderList_train,classifier='Ada')

83.999999999999986

In [22]:
cv(feat_vec_train_2,genderList_train,classifier="Ada")

83.166666666666671

### RandomForest classifier

In [23]:
cv(feat_vec_train,genderList_train,classifier='RF')

82.333333333333329

In [24]:
cv(feat_vec_train_2,genderList_train,classifier="RF")

82.0

In [17]:
cv(dtMatTrain,genderList_train,classifier='LR')

74.0

## SVD Decomposition

In [25]:


def tSVD(Train_Mat,it):
		svd = TruncatedSVD(n_components=it, n_iter=7, random_state=42)
		svd.fit(Train_Mat)
		DT_tSVD = svd.transform(Train_Mat)
		return DT_tSVD


In [26]:
####GRID SEARCH METHOD FOR FINDING N_COMPONENT

#for it in range(100,500,50):
#	print it
it = 300
DT_tSVD = tSVD(feat_vec_train_2,it)
print (DT_tSVD).shape

TD_SVD = cv(DT_tSVD , genderList_train,classifier='SVM')
print 'Avg accuracy_SVM :',TD_SVD

(600, 300)
Avg accuracy_SVM : 79.0


In [28]:
TD_SVD_DT = cv(DT_tSVD , genderList_train,classifier='DT')
print 'Avg accuracy_DT :',TD_SVD_DT

Avg accuracy_DT : 56.0


In [29]:
TD_SVD_Ada = cv(DT_tSVD , genderList_train,classifier='Ada')
print 'Avg accuracy_SVM_Ada :',TD_SVD_Ada

Avg accuracy_SVM_Ada : 71.5


In [30]:
TD_SVD_RF = cv(DT_tSVD , genderList_train,classifier='RF')
print 'Avg accuracy_RF :',TD_SVD_RF

Avg accuracy_RF : 72.1666666667
